In [17]:
from pyscf import gto, scf, cc
import numpy as np
from jax import numpy as jnp
from jax import vmap, jvp, jit
import jax
from functools import partial

a = 1.05835 # 2aB
nH = 2
atoms = ""
for i in range(nH):
    atoms += f"O {i*a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="crenbl", spin=2, verbose=4)
mol.build()

mf = scf.UHF(mol) #.density_fit()
mf.kernel()

mo,_ = mf.stability()
dm = mf.make_rdm1(mo,mo_occ=mf.mo_occ)
mf.kernel(dm0=dm)

nfrozen = 0
mycc = cc.CCSD(mf,frozen=nfrozen)
e = mycc.kernel()

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#5074-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Sun Oct 12 21:35:37 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 16
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 2
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 O      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.0

In [ ]:
# norb_frozen = 2
# overlap = mf.get_ovlp()
# # represent mo_beta in mo_alpha
# # |beta> = |alpha_p><alpha_p|beta_q>
# # c_pq = <alpha|beta> = mo^a_pr <ao_r|ao_s> mo^b_sq
# c_b_in_a = mf.mo_coeff[0].T @ overlap @ mf.mo_coeff[1][:,norb_frozen:]
# print(c_b_in_a.shape)

(32, 30)


In [70]:
from ad_afqmc.cisd_perturb import ccsd_pt
options = {'n_eql': 4,
           'n_prop_steps': 50,
            'n_ene_blocks': 20,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 2,
            'seed': 2,
            'walker_type': 'uhf',
            'trial': 'ucisd',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

from ad_afqmc import pyscf_interface
from ad_afqmc.cisd_perturb import sample_pt2
pyscf_interface.prep_afqmc(mycc,chol_cut=1e-7)
# prep_afqmc(mycc,chol_cut=1e-7,norb_frozen=2)

from ad_afqmc import config, mpi_jax, wavefunctions
import time
from jax import random
ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ \
    = (mpi_jax._prep_afqmc(options))

#
# Preparing AFQMC calculation
# If you import pyscf cc modules and use MPI for AFQMC in the same script, finalize MPI before calling the AFQMC driver.
# Calculating Cholesky integrals
# Finished calculating Cholesky integrals
#
# Size of the correlation space:
# Number of electrons: (9, 7)
# Number of basis functions: 32
# Number of Cholesky vectors: 221
#
# Number of MPI ranks: 1
#
# norb: 32
# nelec: (9, 7)
#
# n_eql: 4
# n_prop_steps: 50
# n_ene_blocks: 20
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 2
# seed: 2
# walker_type: uhf
# trial: ucisd
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#


In [71]:
config.setup_jax()
MPI = config.setup_comm()
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

seed = options['seed']

# mo_coeff = wave_data["mo_coeff"]
# wave_data["mo_coeff"] = [
#     mo_coeff[0][:, : trial.nelec[0]],
#     mo_coeff[1][:, : trial.nelec[1]],
# ]

ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(
    ham_data, prop, trial, wave_data
)
prop_data = prop.init_prop_data(trial, wave_data, ham_data, None)
prop_data["key"] = random.PRNGKey(seed + rank)
print('init walker energy: ',prop_data['e_estimate'])
print('CCSD energy: ',mycc.e_tot)

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
init walker energy:  -122.58687008196475
CCSD energy:  -122.58687045728139


In [72]:
mo_coeff = wave_data["mo_coeff"]
wave_data_uhf = wave_data
wave_data_uhf["mo_coeff"] = [
    mo_coeff[0][:, : trial.nelec[0]],
    mo_coeff[1][:, : trial.nelec[1]],
]

In [73]:
guide = wavefunctions.uhf(trial.norb,trial.nelec,trial.n_batch)
olps = guide.calc_overlap(prop_data['walkers'],wave_data_uhf)

In [68]:
from jax import lax, jit, vmap
from functools import partial

@partial(jit, static_argnums=4)
def _uccsd_walker_energy_pt(
    walker_up: jax.Array,
    walker_dn: jax.Array,
    ham_data: dict,
    wave_data: dict,
    trial,
) -> complex:
    nocc_a, ci1_a, ci2_aa = trial.nelec[0], wave_data["ci1A"], wave_data["ci2AA"]
    nocc_b, ci1_b, ci2_bb = trial.nelec[1], wave_data["ci1B"], wave_data["ci2BB"]
    ci2_ab = wave_data["ci2AB"]
    walker_dn_b = wave_data["mo_coeff"][1].T.dot(walker_dn[:, :nocc_b])
    green_a = (walker_up.dot(jnp.linalg.inv(walker_up[:nocc_a, :]))).T
    green_b = (walker_dn_b.dot(jnp.linalg.inv(walker_dn_b[:nocc_b, :]))).T
    green_occ_a = green_a[:, nocc_a:].copy()
    green_occ_b = green_b[:, nocc_b:].copy()
    greenp_a = jnp.vstack((green_occ_a, -jnp.eye(trial.norb - nocc_a)))
    greenp_b = jnp.vstack((green_occ_b, -jnp.eye(trial.norb - nocc_b)))

    chol_a = ham_data["chol"].reshape(-1, trial.norb, trial.norb)
    chol_b = ham_data["chol_b"].reshape(-1, trial.norb, trial.norb)
    rot_chol_a = chol_a[:, :nocc_a, :]
    rot_chol_b = chol_b[:, :nocc_b, :]
    h1_a = (ham_data["h1"][0] + ham_data["h1"][1]) / 2.0
    h1_b = ham_data["h1_b"]
    hg_a = jnp.einsum("pj,pj->", h1_a[:nocc_a, :], green_a)
    hg_b = jnp.einsum("pj,pj->", h1_b[:nocc_b, :], green_b)
    hg = hg_a + hg_b

    # 0 body energy
    h0 = ham_data["h0"]

    # 1 body energy
    # ref
    e1_0 = hg

    # single excitations
    ci1g_a = jnp.einsum("pt,pt->", ci1_a, green_occ_a, optimize="optimal")
    ci1g_b = jnp.einsum("pt,pt->", ci1_b, green_occ_b, optimize="optimal")
    ci1g = ci1g_a + ci1g_b
    e1_1_1 = ci1g * hg
    gpci1_a = greenp_a @ ci1_a.T
    gpci1_b = greenp_b @ ci1_b.T
    ci1_green_a = gpci1_a @ green_a
    ci1_green_b = gpci1_b @ green_b
    e1_1_2 = -(
        jnp.einsum("ij,ij->", h1_a, ci1_green_a, optimize="optimal")
        + jnp.einsum("ij,ij->", h1_b, ci1_green_b, optimize="optimal")
    )
    e1_1 = e1_1_1 + e1_1_2

    # double excitations
    ci2g_a = jnp.einsum("ptqu,pt->qu", ci2_aa, green_occ_a) / 4
    ci2g_b = jnp.einsum("ptqu,pt->qu", ci2_bb, green_occ_b) / 4
    ci2g_ab_a = jnp.einsum("ptqu,qu->pt", ci2_ab, green_occ_b)
    ci2g_ab_b = jnp.einsum("ptqu,pt->qu", ci2_ab, green_occ_a)
    gci2g_a = jnp.einsum("qu,qu->", ci2g_a, green_occ_a, optimize="optimal")
    gci2g_b = jnp.einsum("qu,qu->", ci2g_b, green_occ_b, optimize="optimal")
    gci2g_ab = jnp.einsum("pt,pt->", ci2g_ab_a, green_occ_a, optimize="optimal")
    gci2g = 2 * (gci2g_a + gci2g_b) + gci2g_ab
    e1_2_1 = hg * gci2g
    ci2_green_a = (greenp_a @ ci2g_a.T) @ green_a
    ci2_green_ab_a = (greenp_a @ ci2g_ab_a.T) @ green_a
    ci2_green_b = (greenp_b @ ci2g_b.T) @ green_b
    ci2_green_ab_b = (greenp_b @ ci2g_ab_b.T) @ green_b
    e1_2_2_a = -jnp.einsum(
        "ij,ij->", h1_a, 4 * ci2_green_a + ci2_green_ab_a, optimize="optimal"
    )
    e1_2_2_b = -jnp.einsum(
        "ij,ij->", h1_b, 4 * ci2_green_b + ci2_green_ab_b, optimize="optimal"
    )
    e1_2_2 = e1_2_2_a + e1_2_2_b
    e1_2 = e1_2_1 + e1_2_2

    # e1 = e1_0 + e1_1 + e1_2

    # two body energy
    # ref
    lg_a = jnp.einsum("gpj,pj->g", rot_chol_a, green_a, optimize="optimal")
    lg_b = jnp.einsum("gpj,pj->g", rot_chol_b, green_b, optimize="optimal")
    e2_0_1 = ((lg_a + lg_b) @ (lg_a + lg_b)) / 2.0
    lg1_a = jnp.einsum("gpj,qj->gpq", rot_chol_a, green_a, optimize="optimal")
    lg1_b = jnp.einsum("gpj,qj->gpq", rot_chol_b, green_b, optimize="optimal")
    e2_0_2 = (
        -(
            jnp.sum(vmap(lambda x: x * x.T)(lg1_a))
            + jnp.sum(vmap(lambda x: x * x.T)(lg1_b))
        )
        / 2.0
    )
    e2_0 = e2_0_1 + e2_0_2

    # single excitations
    e2_1_1 = e2_0 * ci1g
    lci1g_a = jnp.einsum("gij,ij->g", chol_a, ci1_green_a, optimize="optimal")
    lci1g_b = jnp.einsum("gij,ij->g", chol_b, ci1_green_b, optimize="optimal")
    e2_1_2 = -((lci1g_a + lci1g_b) @ (lg_a + lg_b))
    ci1g1_a = ci1_a @ green_occ_a.T
    ci1g1_b = ci1_b @ green_occ_b.T
    e2_1_3_1 = jnp.einsum(
        "gpq,gqr,rp->", lg1_a, lg1_a, ci1g1_a, optimize="optimal"
    ) + jnp.einsum("gpq,gqr,rp->", lg1_b, lg1_b, ci1g1_b, optimize="optimal")
    lci1g_a = jnp.einsum(
        "gip,qi->gpq", ham_data["lci1_a"], green_a, optimize="optimal"
    )
    lci1g_b = jnp.einsum(
        "gip,qi->gpq", ham_data["lci1_b"], green_b, optimize="optimal"
    )
    e2_1_3_2 = -jnp.einsum(
        "gpq,gqp->", lci1g_a, lg1_a, optimize="optimal"
    ) - jnp.einsum("gpq,gqp->", lci1g_b, lg1_b, optimize="optimal")
    e2_1_3 = e2_1_3_1 + e2_1_3_2
    e2_1 = e2_1_1 + e2_1_2 + e2_1_3

    # double excitations
    e2_2_1 = e2_0 * gci2g
    lci2g_a = jnp.einsum(
        "gij,ij->g",
        chol_a,
        8 * ci2_green_a + 2 * ci2_green_ab_a,
        optimize="optimal",
    )
    lci2g_b = jnp.einsum(
        "gij,ij->g",
        chol_b,
        8 * ci2_green_b + 2 * ci2_green_ab_b,
        optimize="optimal",
    )
    e2_2_2_1 = -((lci2g_a + lci2g_b) @ (lg_a + lg_b)) / 2.0

    def scanned_fun(carry, x):
        chol_a_i, rot_chol_a_i, chol_b_i, rot_chol_b_i = x
        gl_a_i = jnp.einsum("pj,ji->pi", green_a, chol_a_i, optimize="optimal")
        gl_b_i = jnp.einsum("pj,ji->pi", green_b, chol_b_i, optimize="optimal")
        lci2_green_a_i = jnp.einsum(
            "pi,ji->pj",
            rot_chol_a_i,
            8 * ci2_green_a + 2 * ci2_green_ab_a,
            optimize="optimal",
        )
        lci2_green_b_i = jnp.einsum(
            "pi,ji->pj",
            rot_chol_b_i,
            8 * ci2_green_b + 2 * ci2_green_ab_b,
            optimize="optimal",
        )
        carry[0] += 0.5 * (
            jnp.einsum("pi,pi->", gl_a_i, lci2_green_a_i, optimize="optimal")
            + jnp.einsum("pi,pi->", gl_b_i, lci2_green_b_i, optimize="optimal")
        )
        glgp_a_i = jnp.einsum(
            "pi,it->pt", gl_a_i, greenp_a, optimize="optimal"
        ).astype(jnp.complex64)
        glgp_b_i = jnp.einsum(
            "pi,it->pt", gl_b_i, greenp_b, optimize="optimal"
        ).astype(jnp.complex64)
        l2ci2_a = 0.5 * jnp.einsum(
            "pt,qu,ptqu->",
            glgp_a_i,
            glgp_a_i,
            ci2_aa.astype(jnp.float32),
            optimize="optimal",
        )
        l2ci2_b = 0.5 * jnp.einsum(
            "pt,qu,ptqu->",
            glgp_b_i,
            glgp_b_i,
            ci2_bb.astype(jnp.float32),
            optimize="optimal",
        )
        l2ci2_ab = jnp.einsum(
            "pt,qu,ptqu->",
            glgp_a_i,
            glgp_b_i,
            ci2_ab.astype(jnp.float32),
            optimize="optimal",
        )
        carry[1] += l2ci2_a + l2ci2_b + l2ci2_ab
        return carry, 0.0

    [e2_2_2_2, e2_2_3], _ = lax.scan(
        scanned_fun, [0.0, 0.0], (chol_a, rot_chol_a, chol_b, rot_chol_b)
    )
    e2_2_2 = e2_2_2_1 + e2_2_2_2
    e2_2 = e2_2_1 + e2_2_2 + e2_2_3

    # e2 = e2_0 + e2_1 + e2_2

    # overlap
    overlap_1 = ci1g  # jnp.einsum("ia,ia", ci1, green_occ)
    overlap_2 = gci2g
    # overlap = 1.0 + overlap_1 + overlap_2

    e1 = e1_0 + e1_1 + e1_2
    e2 = e2_0 + e2_1 + e2_2
    # e_ccsd = (e1 + e2) / overlap + h0

    t = overlap_1 + overlap_2 # <psi|(t1+t2)|phi>/<psi|phi>
    e_0 = e1_0 + e2_0 # <psi|h1+h2|phi>/<psi|phi>
    e_12 = e1_1 + e1_2 + e2_1 + e2_2 # <psi|(t1+t2)(h1+h2)|phi>/<psi|phi>

    return jnp.real(t), jnp.real(e_0), jnp.real(e_12) # doesn't contain h0!!!

@partial(jit, static_argnums=3)
def uccsd_walker_energy_pt(walkers,ham_data,wave_data,trial):
    t, e0, e1 = vmap(
        _uccsd_walker_energy_pt,in_axes=(0, 0, None, None, None))(
        walkers[0], walkers[1], ham_data, wave_data,trial)
    return t, e0, e1

In [69]:
walker_up = prop_data['walkers'][0][0]
walker_dn = prop_data['walkers'][1][0]
h0 = ham_data['h0']
t,e_0,e_12 = _uccsd_walker_energy_pt(walker_up,walker_dn,ham_data,wave_data,trial)
print(t)
print(e_0+h0)
print(e_12)
# print(e_tot)

TypeError: Cannot concatenate arrays with shapes that differ in dimensions other than the one being concatenated: concatenating along dimension 0 for shapes (7, 0), (25, 25).

In [50]:
t,e_0,e_12 = uccsd_walker_energy_pt(prop_data['walkers'],ham_data,wave_data,trial)

In [51]:
print(t)
print(e_0+h0)
print(e_12)

[-3.59689196e-18 -3.59689196e-18]
[-122.2959382 -122.2959382]
[-0.29093188 -0.29093188]


In [ ]:
ci1_a, ci2_aa = wave_data["ci1A"], wave_data["ci2AA"]
ci1_b, ci2_bb = wave_data["ci1B"], wave_data["ci2BB"]
ci2_ab = wave_data["ci2AB"]

In [ ]:
ci2aa = ci2_aa + 2 * np.einsum("ia,jb->ijab", cc.t1[0], cc.t1[0])
ci2aa = (ci2aa - ci2aa.transpose(0, 1, 3, 2)) / 2
ci2aa = ci2aa.transpose(0, 2, 1, 3)
ci2bb = cc.t2[2] + 2 * np.einsum("ia,jb->ijab", cc.t1[1], cc.t1[1])
ci2bb = (ci2bb - ci2bb.transpose(0, 1, 3, 2)) / 2
ci2bb = ci2bb.transpose(0, 2, 1, 3)
ci2ab = cc.t2[1] + np.einsum("ia,jb->ijab", cc.t1[0], cc.t1[1])
ci2ab = ci2ab.transpose(0, 2, 1, 3)
ci1a = np.array(cc.t1[0])
ci1b = np.array(cc.t1[1])

In [ ]:
def prep_afqmc(
    mf_or_cc: Union[scf.uhf.UHF, scf.rhf.RHF, CCSD, UCCSD],
    basis_coeff: Optional[np.ndarray] = None,
    norb_frozen: int = 0,
    chol_cut: float = 1e-5,
    integrals: Optional[dict] = None,
    mo_file = "mo_coeff.npz",
    amp_file = "amplitudes.npz",
    chol_file = "FCIDUMP_chol"
):
    """Prepare AFQMC calculation with mean field trial wavefunction. Writes integrals and mo coefficients to disk.

    Args:
        mf (Union[scf.uhf.UHF, scf.rhf.RHF, mcscf.mc1step.CASSCF]): pyscf mean field object. Used for generating integrals (if not provided) and trial.
        basis_coeff (np.ndarray, optional): Orthonormal basis used for afqmc, given in the basis of ao's. If not provided mo_coeff of mf is used as the basis.
        norb_frozen (int, optional): Number of frozen orbitals. Not supported for custom integrals.
        chol_cut (float, optional): Cholesky decomposition cutoff.
        integrals (dict, optional): Dictionary of integrals in an orthonormal basis, {"h0": enuc, "h1": h1e, "h2": eri}.
    """

    print("#\n# Preparing AFQMC calculation")

    if isinstance(mf_or_cc, (CCSD, UCCSD)):
        # raise warning about mpi
        print(
            "# If you import pyscf cc modules and use MPI for AFQMC in the same script, finalize MPI before calling the AFQMC driver."
        )
        mf = mf_or_cc._scf
        cc = mf_or_cc
        if cc.frozen is not None:
            norb_frozen = cc.frozen
        if isinstance(cc, UCCSD):
            ci2aa = cc.t2[0] #+ 2 * np.einsum("ia,jb->ijab", cc.t1[0], cc.t1[0])
            ci2aa = (ci2aa - ci2aa.transpose(0, 1, 3, 2)) / 2
            ci2aa = ci2aa.transpose(0, 2, 1, 3)
            ci2bb = cc.t2[2] #+ 2 * np.einsum("ia,jb->ijab", cc.t1[1], cc.t1[1])
            ci2bb = (ci2bb - ci2bb.transpose(0, 1, 3, 2)) / 2
            ci2bb = ci2bb.transpose(0, 2, 1, 3)
            ci2ab = cc.t2[1] #+ np.einsum("ia,jb->ijab", cc.t1[0], cc.t1[1])
            ci2ab = ci2ab.transpose(0, 2, 1, 3)
            ci1a = np.array(cc.t1[0])
            ci1b = np.array(cc.t1[1])
            
            np.savez(
                amp_file,
                ci1a=ci1a,
                ci1b=ci1b,
                ci2aa=ci2aa,
                ci2ab=ci2ab,
                ci2bb=ci2bb,
            )
        else:
            ci2 = cc.t2 # + np.einsum("ia,jb->ijab", np.array(cc.t1), np.array(cc.t1))
            ci2 = ci2.transpose(0, 2, 1, 3)
            ci1 = np.array(cc.t1)
            np.savez(amp_file, ci1=ci1, ci2=ci2)
    else:
        mf = mf_or_cc

    mol = mf.mol
    # choose the orbital basis
    if basis_coeff is None:
        if isinstance(mf, scf.uhf.UHF):
            basis_coeff = mf.mo_coeff[0]
        else:
            basis_coeff = mf.mo_coeff

    # calculate cholesky integrals
    print("# Calculating Cholesky integrals")
    h1e, chol, nelec, enuc, nbasis, nchol = [None] * 6
    DFbas = None
    if getattr(mf, "with_df", None) is not None:
        print('# Decomposing ERI with DF')
        DFbas = mf.with_df.auxmol.basis  # type: ignore
    h1e, chol, nelec, enuc = generate_integrals(
        mol, mf.get_hcore(), basis_coeff, chol_cut, DFbas=DFbas
    ) 
    nbasis = h1e.shape[-1]
    nelec = mol.nelec

    if norb_frozen > 0:
        assert norb_frozen * 2 < sum(
            nelec
        ), "Frozen orbitals exceed number of electrons"
        mc = mcscf.CASSCF(
            mf, mol.nao - norb_frozen, mol.nelectron - 2 * norb_frozen
        )
        nelec = mc.nelecas  # type: ignore
        mc.mo_coeff = basis_coeff  # type: ignore
        h1e, enuc = mc.get_h1eff()  # type: ignore
        chol = chol.reshape((-1, nbasis, nbasis))
        chol = chol[:, mc.ncore : mc.ncore + mc.ncas, mc.ncore : mc.ncore + mc.ncas]  # type: ignore

    print("# Finished calculating Cholesky integrals\n#")

    nbasis = h1e.shape[-1]
    print("# Size of the correlation space:")
    print(f"# Number of electrons: {nelec}")
    print(f"# Number of basis functions: {nbasis}")
    print(f"# Number of Cholesky vectors: {chol.shape[0]}\n#")
    chol = chol.reshape((-1, nbasis, nbasis))
    v0 = 0.5 * np.einsum("nik,njk->ij", chol, chol, optimize="optimal")
    h1e_mod = h1e - v0
    chol = chol.reshape((chol.shape[0], -1))

    # write trial mo coefficients
    trial_coeffs = np.empty((2, nbasis, nbasis))
    overlap = mf.get_ovlp(mol)
    if isinstance(mf, (scf.uhf.UHF, scf.rohf.ROHF)):
        uhfCoeffs = np.empty((nbasis, 2 * nbasis))
        if isinstance(mf, scf.uhf.UHF):
            q, r = np.linalg.qr(
                basis_coeff[:, norb_frozen:]
                .T.dot(overlap)
                .dot(mf.mo_coeff[0][:, norb_frozen:])
            )
            sgn = np.sign(r.diagonal())
            q = np.einsum("ij,j->ij", q, sgn)
            # q2 = basis_coeff[:, norb_frozen:].T.dot(overlap).dot(mf.mo_coeff[0][:, norb_frozen:])
            # print("max err a", np.max(abs(q-q2)))
            # q, _ = np.linalg.qr(
            #    basis_coeff[:, norb_frozen:]
            #    .T.dot(overlap)
            #    .dot(mf.mo_coeff[0][:, norb_frozen:])
            # )
            uhfCoeffs[:, :nbasis] = q
            q, r = np.linalg.qr(
                basis_coeff[:, norb_frozen:]
                .T.dot(overlap)
                .dot(mf.mo_coeff[1][:, norb_frozen:])
            )
            sgn = np.sign(r.diagonal())
            q = np.einsum("ij,j->ij", q, sgn)
            # q2 = basis_coeff[:, norb_frozen:].T.dot(overlap).dot(mf.mo_coeff[1][:, norb_frozen:])
            # print("max err b", np.max(abs(q-q2)))
            # import pdb
            # pdb.set_trace()
            # q, _ = np.linalg.qr(
            #     basis_coeff[:, norb_frozen:]
            #     .T.dot(overlap)
            #     .dot(mf.mo_coeff[1][:, norb_frozen:])
            # )
            uhfCoeffs[:, nbasis:] = q
        else:
            q, r = np.linalg.qr(
                basis_coeff[:, norb_frozen:]
                .T.dot(overlap)
                .dot(mf.mo_coeff[:, norb_frozen:])
            )
            sgn = np.sign(r.diagonal())
            q = np.einsum("ij,j->ij", q, sgn)
            uhfCoeffs[:, :nbasis] = q
            uhfCoeffs[:, nbasis:] = q

        trial_coeffs[0] = uhfCoeffs[:, :nbasis]
        trial_coeffs[1] = uhfCoeffs[:, nbasis:]
        # np.savetxt("uhf.txt", uhfCoeffs)
        np.savez(mo_file, mo_coeff=trial_coeffs)

    elif isinstance(mf, scf.rhf.RHF):
        q, _ = np.linalg.qr(
            basis_coeff[:, norb_frozen:]
            .T.dot(overlap)
            .dot(mf.mo_coeff[:, norb_frozen:])
        )
        trial_coeffs[0] = q
        trial_coeffs[1] = q
        np.savez(mo_file, mo_coeff=trial_coeffs)
    
    write_dqmc(
        h1e,
        h1e_mod,
        chol,
        sum(nelec),
        nbasis,
        enuc,
        ms=mol.spin,
        filename=chol_file,
        mo_coeffs=trial_coeffs,
    )

In [ ]:
norb = trial.norb
nocc = wave_data['ci1'].shape[0]
wave_data["mo_coeff"] = np.eye(norb)[:,:nocc]
ci1,ci2 = wave_data['ci1'],wave_data['ci2']
t2 = ci2 - jnp.einsum('ia,jb->iajb',ci1,ci1)
wave_data['t1'] = ci1
wave_data['t2'] = t2

In [40]:
trial._calc_energy(walker_up,walker_dn,ham_data,wave_data)

Array(-122.58687008+0.j, dtype=complex128)

In [32]:
(e_0 + e_12)/(1+t)+h0

Array(-122.58687008+0.j, dtype=complex128)

In [26]:
print(mf.e_tot)

-122.29593837787019


In [ ]:
from ad_afqmc.cisd_perturb import sample_pt2
prop_data["n_killed_walkers"] = 0

prop_data, (blk_wt, blk_t, blk_e0, blk_e1) = \
    sample_pt2._block_scan(prop_data,ham_data,prop,trial,wave_data,sampler)